In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime



In [5]:
import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, AGE, RANK AS COMPETITION_RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, DATE
FROM `saa-analytics.results.saa_full` 
WHERE RESULT!='NM' AND RESULT!='FOUL' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT IS NOT NULL AND (CATEGORY_EVENT='Long' OR CATEGORY_EVENT='Sprint' OR CATEGORY_EVENT='Mid' OR CATEGORY_EVENT='Hurdles' OR CATEGORY_EVENT='Relay' OR CATEGORY_EVENT='Walk' OR CATEGORY_EVENT='Marathon' OR CATEGORY_EVENT='Steeple')
"""

athletes = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [6]:
athletes

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,DATE
0,Supanich Poolkerd,11.63,None,1,100m,Sprint,Female,SEA Games,None
1,Kristina Knott,11.66,None,2,100m,Sprint,Female,SEA Games,None
2,Zaidatul Husniah Zulkifli,11.69,None,3,100m,Sprint,Female,SEA Games,None
3,Hoang Du Y,11.73,None,4,100m,Sprint,Female,SEA Games,None
4,Elizabeth-Ann Tan Shee Ru,12.0,None,5,100m,Sprint,Female,SEA Games,None
...,...,...,...,...,...,...,...,...,...
17359,"Aw, Hailey",37.68,11,28,Girls 11-12 200 Meter Dash U13,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
17360,"Wang, Minqi",29.97,12,4,Girls 11-12 200 Meter Dash U13,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
17361,"Koh, Jayna Jia Y",33.03,12,19,Girls 11-12 200 Meter Dash U13,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
17362,"Mak, Chen Yu, Adelyn",34.73,11,25,Girls 11-12 200 Meter Dash U13,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023


In [7]:
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+200 Meter Dash.+', value='200m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+100 Meter Dash.+', value='100m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400 Meter Dash.+', value='400m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+800 Meter Run.+', value='800m')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500 Meter Run.+', value='1500m run')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Run.+', value='3000m run')


athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x100 Meter Relay.+', value='4 x 100m relay')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x400 Meter Relay.+', value='4 x 400m relay')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 meter.+', value='3000m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Race Walk.+', value='5000m race walk')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Run.+', value='5000m run')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+10000 Meter Run.+', value='10000m run')


athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000m Race Walk.+', value='5000m race walk')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000m Race Walk.+', value='3000m race walk')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500m Race Walk.+', value='1500m race walk')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Pole Vault.+', value='Pole vault')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'Men Shot Put OPEN', value='Shot put')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'Women Shot Put U18', value='Shot put')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'Men Discus Throw U18', value='Discus throw')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'Men Discus Throw OPEN', value='Discus throw')


athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Triple Jump.+', value='Triple jump')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Javelin Throw Open', value='Javelin throw')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Javelin Throw Open', value='Javelin throw')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Long Jump Open.+', value='Long jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'Long Jump', value='Long jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Long Jump U15.+', value='Long jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Long Jump U18.+', value='Long jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Long Jump Men.+', value='Long jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Long Jump U20.+', value='Long jump')




athletes['EVENT'] = athletes['EVENT'].replace(regex=r'High Jump', value='High jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump Novice.+', value='High jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump Intermediate.+', value='High jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump Open.+', value='High jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump U17.+', value='High jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump U15.+', value='High jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump U13.+', value='High jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump Masters.+', value='High jump')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+S/C.+', value='steeplechase')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Steeplechase (0.914m).+', value='3000m steeplechase (0.914m)')




In [8]:
athletes

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,DATE
0,Supanich Poolkerd,11.63,None,1,100m,Sprint,Female,SEA Games,None
1,Kristina Knott,11.66,None,2,100m,Sprint,Female,SEA Games,None
2,Zaidatul Husniah Zulkifli,11.69,None,3,100m,Sprint,Female,SEA Games,None
3,Hoang Du Y,11.73,None,4,100m,Sprint,Female,SEA Games,None
4,Elizabeth-Ann Tan Shee Ru,12.0,None,5,100m,Sprint,Female,SEA Games,None
...,...,...,...,...,...,...,...,...,...
17359,"Aw, Hailey",37.68,11,28,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
17360,"Wang, Minqi",29.97,12,4,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
17361,"Koh, Jayna Jia Y",33.03,12,19,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
17362,"Mak, Chen Yu, Adelyn",34.73,11,25,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023


In [10]:
credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
)

sql="""
SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
FROM `saa-analytics.results.saa_full`
WHERE STAGE='Final' AND COMPETITION='SEA Games' AND RANK='3' AND (CATEGORY_EVENT='Sprint' OR CATEGORY_EVENT='Mid' OR CATEGORY_EVENT='Hurdles' OR CATEGORY_EVENT='Relay' OR CATEGORY_EVENT='Walk' OR CATEGORY_EVENT='Marathon' OR CATEGORY_EVENT='Steeple')
"""

benchmarks = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [11]:
benchmarks

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE
0,Trần Thị Nhi Yến,11.75,3,100m,Sprint,Female,SEA Games,Final
1,Zaidatul Husniah Zulkifli,23.6,3,200m,Sprint,Female,SEA Games,Final
2,Nguyễn Thị Hằng,53.84,3,400m,Sprint,Female,SEA Games,Final
3,Goh Chui Ling,2:09.15,3,800m,Mid,Female,SEA Games,Final
4,Goh Chui Ling,4:26.33,3,1500m,Mid,Female,SEA Games,Final
5,Odekta Elvina Naibaho,35:31.03,3,10000m,Sprint,Female,SEA Games,Final
6,Dina Aulia,13.59,3,100m hurdles,Hurdles,Female,SEA Games,Final
7,Nguyễn Thị Ngọc,59.09,3,400m hurdles,Hurdles,Female,SEA Games,Final
8,Nguyễn Thị Huong,11:00.85,3,3000m steeplechase,Steeple,Female,SEA Games,Final
9,"Azreen Nabila Alias, Nur Afrina Batrisyia, N...",44.58,3,4 x 100m relay,Relay,Female,SEA Games,Final


In [12]:
benchmarks.rename(columns = {'RESULT':'BENCHMARK'}, inplace = True)


In [13]:
benchmarks.drop(['NAME', 'RANK', 'CATEGORY_EVENT', 'COMPETITION', 'STAGE'], axis=1, inplace=True)


In [64]:
benchmarks

,BENCHMARK,EVENT,GENDER
0,11.75,100m,Female
1,23.6,200m,Female
2,53.84,400m,Female
3,2:09.15,800m,Female
4,4:26.33,1500m,Female
5,35:31.03,10000m,Female
6,13.59,100m hurdles,Female
7,59.09,400m hurdles,Female
8,11:00.85,3000m steeplechase,Female
9,44.58,4 x 100m relay,Female


In [18]:
def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()    
    
    if any(s in string for s in l)==True:
    
        output=float(str(metric))
        
    else:
        
        searchstring = ":"
        searchstring2 = "."
        substring=str(metric)
        count = substring.count(searchstring)
        count2 = substring.count(searchstring2)
            
        if count==0:
            output=float(substring)
                        
             
        elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                                                
            time=str(metric)
            h, m ,s = time.split(':')
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
        elif (count==1 and count2==1):
            
            m,s = metric.split(':')
            output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
            
            if output==229.90:
                print(metric, m, s, output, 'here')

                     
        elif (count==1 and count2==2):
            
            metric = metric.replace(".", ":", 1)
            
            h,m,s = metric.split(':')            
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
        elif (count==2 and count2==0):
            
            h,m,s = metric.split(':')
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                
    return output

In [16]:
for i in range(len(benchmarks)):
        
    rowIndex = benchmarks.index[i]

    input_string=benchmarks.iloc[rowIndex,1]
    metric=benchmarks.iloc[rowIndex,0]
        
    out = convert_time(i, input_string, metric)
     
    benchmarks.loc[rowIndex, 'Metric'] = out

In [17]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric
0,11.75,100m,Female,11.750
1,23.6,200m,Female,23.600
2,53.84,400m,Female,53.840
3,2:09.15,800m,Female,129.150
4,4:26.33,1500m,Female,266.330
5,35:31.03,10000m,Female,2131.030
6,13.59,100m hurdles,Female,13.590
7,59.09,400m hurdles,Female,59.090
8,11:00.85,3000m steeplechase,Female,660.850
9,44.58,4 x 100m relay,Female,44.580


In [19]:
benchmarks['2pc']=benchmarks['Metric']*1.02
benchmarks['35pc']=benchmarks['Metric']*1.035
benchmarks['5pc']=benchmarks['Metric']*1.05

In [20]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric,2pc,35pc,5pc
0,11.75,100m,Female,11.750,11.98500,12.161250,12.33750
1,23.6,200m,Female,23.600,24.07200,24.426000,24.78000
2,53.84,400m,Female,53.840,54.91680,55.724400,56.53200
3,2:09.15,800m,Female,129.150,131.73300,133.670250,135.60750
4,4:26.33,1500m,Female,266.330,271.65660,275.651550,279.64650
5,35:31.03,10000m,Female,2131.030,2173.65060,2205.616050,2237.58150
6,13.59,100m hurdles,Female,13.590,13.86180,14.065650,14.26950
7,59.09,400m hurdles,Female,59.090,60.27180,61.158150,62.04450
8,11:00.85,3000m steeplechase,Female,660.850,674.06700,683.979750,693.89250
9,44.58,4 x 100m relay,Female,44.580,45.47160,46.140300,46.80900


In [21]:
df = athletes.reset_index().merge(benchmarks.reset_index(), on=['EVENT','GENDER'], how='left')


In [22]:
df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc
0,0,Supanich Poolkerd,11.63,None,1,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375
1,1,Kristina Knott,11.66,None,2,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375
2,2,Zaidatul Husniah Zulkifli,11.69,None,3,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375
3,3,Hoang Du Y,11.73,None,4,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375
4,4,Elizabeth-Ann Tan Shee Ru,12.0,None,5,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17359,17359,"Aw, Hailey",37.68,11,28,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,1.0,23.6,23.60,24.072,24.42600,24.7800
17360,17360,"Wang, Minqi",29.97,12,4,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,1.0,23.6,23.60,24.072,24.42600,24.7800
17361,17361,"Koh, Jayna Jia Y",33.03,12,19,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,1.0,23.6,23.60,24.072,24.42600,24.7800
17362,17362,"Mak, Chen Yu, Adelyn",34.73,11,25,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,1.0,23.6,23.60,24.072,24.42600,24.7800


In [ ]:
mask=